In [1]:
codigo_canhero = 12478

In [3]:
from werkzeug.wrappers import Request, Response
#from flask import Flask, render_template
from jinja2 import Template

#from datetime import date
from datetime import datetime, timedelta

import collections
from amigocloud import AmigoCloud

from datetime import datetime

In [5]:
acceso = open('../api_key/api.key','r')
api_token = acceso.readlines()[0]
acceso.close()

amigocloud = AmigoCloud(token=api_token)
amigocloud

In [8]:
# DATOS DE LAS INPECCION
query = {'query': 'select * from dataset_296354 where cast(split_part(canhero, \' / \', 1) as integer) = {cod_ca}'.format(cod_ca=codigo_canhero)}
project = 'https://app.amigocloud.com/api/v1/projects/32082/sql'
select = amigocloud.get(project, query)
data = select['data'][0]

data['fecha_de_registro'] = datetime.strptime(data['fecha_de_registro'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
#data['fecha_quema'] = datetime.strptime(data['fecha_quema'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')
#data['fecha_inspeccion'] = datetime.strptime(data['fecha_inspeccion'][0:10], '%Y-%m-%d').strftime('%d/%m/%Y')

insp = collections.namedtuple("insp", data.keys())(*data.values())
print('end')

end


In [ ]:
insp

insp(fecha_de_registro='18/07/2022', amigo_id='6a939ebf0f994778852fda507e1e348d', ubicacion='0101000020E61000003DCC03FD2A904FC05F420587174631C0', cod_prop=495, nom_prop='LAS_MARAS--HUGO_SERRATE', cosechadora='Cameco 2500', codigo_maquina='Cc05', nombre_operador='David', tiempo_experiencia=3, op_01=False, op_02=False, op_03=False, op_04=False, op_05=False, op_06=False, op_07=False, op_08=False, op_09=False, op_10=False, op_11=False, op_12=False, op_13=False, op_14=False, op_15=False, op_16=False, op_17=False, op_18=False, op_19=False, op_20=False, op_21=False, op_22=False, op_23=False, maq_01=False, maq_02=False, maq_03=False, maq_04=False, maq_05=False, maq_06=True, maq_07=True, maq_08=True, maq_09=True, maq_10=False, maq_11=True, maq_12=True, maq_13=True, canhero='12478 / SERRATE COLAMARINO HUGO', nota_operador=0.0, nota_maquina=5.38461538461538, op_01_orientacion=None, op_02_orientacion=None, op_03_orientacion=None, op_04_orientacion=None, op_05_orientacion=None, op_06_orientacion=No

In [34]:
query_fotos = {'query': 'select source_amigo_id, s3_filename from gallery_43414 where source_amigo_id=\'{source_amigo_id}\''.format(source_amigo_id=insp.amigo_id)}
project = 'https://app.amigocloud.com/api/v1/projects/32082/sql'
fotos = amigocloud.get(project, query_fotos)['data']

#if fotos_filename['data'] != []:
#    dic = {'lote': lote.lote, 'file_name': fotos_filename['data'][0]['s3_filename']}
#    fotos.append(dic)

#fotos

In [36]:
fotos

[{'source_amigo_id': '6a939ebf0f994778852fda507e1e348d',
  's3_filename': '43414_20220718092030089.jpg'},
 {'source_amigo_id': '6a939ebf0f994778852fda507e1e348d',
  's3_filename': '43414_20220718092123638.jpg'},
 {'source_amigo_id': '6a939ebf0f994778852fda507e1e348d',
  's3_filename': '43414_20220718092101910.jpg'},
 {'source_amigo_id': '6a939ebf0f994778852fda507e1e348d',
  's3_filename': '43414_20220718092151983.jpg'},
 {'source_amigo_id': '6a939ebf0f994778852fda507e1e348d',
  's3_filename': '43414_20220718092219010.jpg'},
 {'source_amigo_id': '6a939ebf0f994778852fda507e1e348d',
  's3_filename': '43414_20220718092254473.jpg'},
 {'source_amigo_id': '6a939ebf0f994778852fda507e1e348d',
  's3_filename': '43414_20220718093419944.jpg'},
 {'source_amigo_id': '6a939ebf0f994778852fda507e1e348d',
  's3_filename': '43414_20220718095050282.jpg'},
 {'source_amigo_id': '6a939ebf0f994778852fda507e1e348d',
  's3_filename': '43414_20220718092453870.jpg'}]

In [71]:
from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

import requests

doc = DocxTemplate("templates/01_template_operador.docx")

#descargar fotos y generar lista InlineImage

lista_fotos_inline = []
for foto in fotos:
    #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['file_name'])
    #response = requests.get("https://www-amigocloud.s3.amazonaws.com/gallery/" + foto['s3_filename'])
    #file = open('fotos/' + foto['s3_filename'], "wb")
    #file.write(response.content)
    #file.close()
    lista_fotos_inline.append({'foto': docxtpl.InlineImage(doc, image_descriptor='fotos/' + foto['s3_filename'], width=Mm(50))})

context = {'insp':insp, 'fotos':lista_fotos_inline}

doc.render(context)

# formato de nombre de archivo: "123_CQ_01-01-2022_NOMBRE"
cod_nom = insp.canhero.split(' / ')
file_name = cod_nom[0] + '_EC_' + insp.fecha_de_registro.replace('/','-') + '_' + cod_nom[1]

doc.save('_' + file_name + '.docx')